In [1]:
import json
from datetime import datetime
from typing import List, Dict

import numpy as np
import tensorflow as tf

import spectrum_painting_data as sp_data
import spectrum_painting_model as sp_model
import spectrum_painting_predict as sp_predict
import spectrum_painting_training as sp_training
from training.result import Result

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
classes = ["Z", "B", "W", "BW", "ZB", "ZW", "ZBW"]
snr_list = [-100, -15, -10, -5, 0, 5, 10, 15, 20, 30]

spectrum_painting_options = sp_training.SpectrumPaintingTrainingOptions(
    downsample_resolution=64,
    k=3,
    l=16,
    d=4
)

training_count = 10

results: List[Dict] = []

for snr in snr_list:
    print(f"Starting snr {snr}")
    print("Loading spectrograms")
    spectrograms = sp_data.load_spectrograms(data_dir="data/numpy",
                                             classes=classes,
                                             snr_list=[snr],
                                             windows_per_spectrogram=256,
                                             window_length=256,
                                             nfft=64)

    full_model_predictions: List[List[int]] = []
    full_model_labels: List[List[int]] = []

    lite_model_predictions: List[List[int]] = []
    lite_model_labels: List[List[int]] = []
    lite_model_sizes: List[int] = []

    lite_model_no_quant_predictions: List[List[int]] = []
    lite_model_no_quant_labels: List[List[int]] = []
    lite_model_no_quant_sizes: List[int] = []
    label_names: [str] = []

    for i in range(training_count):
        print(f"Creating training and test sets. SNR: {snr}, Iteration: {i}")

        train_test_sets = sp_training.create_spectrum_painting_train_test_sets(
            spectrograms=spectrograms,
            label_names=classes,
            options=spectrum_painting_options,
            test_size=0.3
        )
        label_names = train_test_sets.label_names

        image_shape = train_test_sets.x_train_augmented[0].shape

        tf_model = sp_model.create_tensorflow_model(image_shape=image_shape,
                                                    label_count=len(train_test_sets.label_names))

        sp_model.fit_model(tf_model, train_test_sets, epochs=200, early_stopping_patience=20)

        full_model_predictions.append([sp_predict.predict_full_model(tf_model, x_a, x_p) for (x_a, x_p) in
                                       zip(train_test_sets.x_test_augmented, train_test_sets.x_test_painted)])
        full_model_labels.append(train_test_sets.y_test.astype(int).tolist())

        lite_model = sp_model.convert_to_tensorflow_lite(tf_model,
                                                         train_test_sets.x_test_augmented,
                                                         train_test_sets.x_test_painted)

        lite_model_sizes.append(len(lite_model))

        lite_model_predictions.append([sp_predict.predict_lite_model(lite_model, x_a, x_p) for (x_a, x_p) in
                                       zip(train_test_sets.x_test_augmented, train_test_sets.x_test_painted)])
        lite_model_labels.append(train_test_sets.y_test.astype(int).tolist())

        lite_no_quant_model = sp_model.convert_to_tensorflow_lite_no_quantization(tf_model)

        lite_model_no_quant_sizes.append(len(lite_no_quant_model))

        lite_model_no_quant_predictions.append(
            [sp_predict.predict_lite_no_quant_model(lite_no_quant_model, x_a, x_p) for (x_a, x_p) in
             zip(train_test_sets.x_test_augmented, train_test_sets.x_test_painted)])

        lite_model_no_quant_labels.append(train_test_sets.y_test.astype(int).tolist())

    result = Result(
        snr=snr,
        label_names=label_names,
        full_model_labels=full_model_labels,
        full_model_predictions=full_model_predictions,
        lite_model_labels=lite_model_labels,
        lite_model_predictions=lite_model_predictions,
        lite_model_size=np.mean(lite_model_sizes),
        lite_model_no_quant_labels=lite_model_no_quant_labels,
        lite_model_no_quant_predictions=lite_model_no_quant_predictions,
        lite_no_quant_model_size=np.mean(lite_model_no_quant_sizes)
    )
    
    results.append(result.to_dict())

    print("Saving model")
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    with open(f"output/results-{timestamp}.json", "w") as f:
        json.dump({"results": results}, f)


Starting snr -100
Loading spectrograms
Creating training and test sets. SNR: -100, Iteration: 0


2024-06-17 13:51:04.913369: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-06-17 13:51:04.913401: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-17 13:51:04.913416: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-17 13:51:04.913442: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-17 13:51:04.913459: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-06-17 13:51:05.343076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 Epoch: 73, Val. accuracy = 0.12393566966056824INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmp4_yxv2mi/assets


INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmp4_yxv2mi/assets
/Users/sethd/Desktop/spectrum-painting/venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-17 13:52:19.455314: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-17 13:52:19.455326: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-17 13:52:19.455575: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmp4_yxv2mi
2024-06-17 13:52:19.457194: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-17 13:52:19.457199: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/81/3wz7wxg90f3

INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmp9unpgddj/assets


...

Creating training and test sets. SNR: -100, Iteration: 3
 Epoch: 61, Val. accuracy = 0.14191107451915748INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpo5r8i17o/assets


INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpo5r8i17o/assets
/Users/sethd/Desktop/spectrum-painting/venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-17 13:56:37.385120: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-17 13:56:37.385136: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-17 13:56:37.385281: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpo5r8i17o
2024-06-17 13:56:37.387324: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-17 13:56:37.387330: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/81/3wz7wxg90f3

INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpuqycrlfw/assets


INFO:tensorflow:Assets written to: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpuqycrlfw/assets
2024-06-17 13:56:39.254339: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-17 13:56:39.254352: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-17 13:56:39.254450: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpuqycrlfw
2024-06-17 13:56:39.256355: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-17 13:56:39.256360: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/81/3wz7wxg90f3df6qvy1xbf71m0000gn/T/tmpuqycrlfw
2024-06-17 13:56:39.260639: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-17 13:56:39.316025: I tensorflow/cc/saved_model/loader.cc:217] Running initialization

Creating training and test sets. SNR: -100, Iteration: 4
 Epoch: 50, Val. accuracy = 0.14001892507076263